# Declare Notebook parameters

In [1]:
## Specify which scenario this analysis is for
# Only one of the parameters below can be True 
current_summer = False
future_summer = True
future_winter = False

## Specify whether the analysis is for office only
only_office = False

# Import required modules

In [2]:
# Built-in modules
import os

# Third party modules
import pandas as pd
import numpy as np
from collections import OrderedDict
import pylogit as pl

# Local modules
import pylogit as pl
import MNL_estimation as MNL



# Read and pre-process the survey data

In [3]:
os.listdir('.')

['Analysis-Future_winter_pre.ipynb',
 'clean_code.ipynb',
 'FlexBldgData_Round2_2020-05-18._Clean for Hassan.csv',
 '__pycache__',
 '.ipynb_checkpoints',
 'MNL_estimation.py']

In [4]:
df_raw = pd.read_csv('FlexBldgData_Round2_2020-05-18._Clean for Hassan.csv')

# Add a customn individual ID column
df_raw['indiv_id'] = range(1,len(df_raw)+1)

# Change date column to datetime data type
df_raw = df_raw[(pd.to_datetime(df_raw['StartDate']) > '2020-02-03 10:55:23')]
df_raw.shape

(49, 472)

In [5]:
## Qualtrics puts all the responses 
# of an individual into one row. 
# So the first step is 
# to switch to wide format (one row per response) 
# and the long format using pylogit

start_index = 53 ## Index at which the DCE questions start

DCE_i = []

question_initials = ['F-', 'S-', 'W-']

number_of_attributes = [5, 7, 7] ## Number of attributes per scenario year

for scenario in range(3):
    for DCE in range(1,9): ##loop over number of choices
        chosen_alts = df_raw.columns[start_index + DCE + 8*scenario]
        print(chosen_alts)
        cols = ['indiv_id',chosen_alts]
        for alt in range(1,3):
            for attr in range(1,number_of_attributes[scenario]): ## This here is range(1,5) for current summer, and range(1,7) for future scenarios
                column = question_initials[scenario] + str(DCE) + '-' + str(alt) + '-' + str(attr)
                cols.append(column)

        
        DCE_i.append(df_raw[cols].values)
        
DCE_i_current_summer = DCE_i[:8]
DCE_i_future_summer = DCE_i[8:16]
DCE_i_future_winter = DCE_i[16:24]

DCE1-1a_1
DCE1-2a_1
DCE1-3a_1
DCE1-4a_1
DCE1-5a_1
DCE1-6a_1
DCE1-7a_1
DCE1-8a_1
DCE2a-1a_1
DCE2a-2a_1
DCE2a-3a_1
DCE2a-4a_1
DCE2a-5a_1
DCE2a-6a_1
DCE2a-7a_1
DCE2a-8a_1
DCE2b-1a_1
DCE2b-2a_1
DCE2b-3a_1
DCE2b-4a_1
DCE2b-5a_1
DCE2b-6a_1
DCE2b-7a_1
DCE2b-8a_1


In [6]:
## current summer

df_cols = ['indiv_id','Alt_chosen']
for alt in range(1,3):
    for attrib in range(1,5):
        df_cols.append('alt_' + str(alt) + '_att_' + str(attrib))
df_current_summer = pd.DataFrame(np.row_stack(DCE_i_current_summer),columns=df_cols)
df_current_summer['AV_0'] = 1
df_current_summer['AV_1'] = 1

df_current_summer.sort_values('indiv_id',inplace=True)

## future summer

df_cols = ['indiv_id','Alt_chosen']
for alt in range(1,3):
    for attrib in range(1,7):
        df_cols.append('alt_' + str(alt) + '_att_' + str(attrib))
df_future_summer = pd.DataFrame(np.row_stack(DCE_i_future_summer),columns=df_cols)
df_future_summer['AV_0'] = 1
df_future_summer['AV_1'] = 1

df_future_summer.sort_values('indiv_id',inplace=True)

## future winter
df_cols = ['indiv_id','Alt_chosen']
for alt in range(1,3):
    for attrib in range(1,7):
        df_cols.append('alt_' + str(alt) + '_att_' + str(attrib))
df_future_winter = pd.DataFrame(np.row_stack(DCE_i_future_winter),columns=df_cols)
df_future_winter['AV_0'] = 1
df_future_winter['AV_1'] = 1

df_future_winter.sort_values('indiv_id',inplace=True)


# Select appropriate scenario dataframe

In [7]:
if current_summer:
    df_scenario = df_current_summer.copy()
elif future_summer:
    df_scenario = df_future_summer.copy()
elif future_winter:
    df_scenario = df_future_winter.copy()
else:
    print("Error: No analysis scenario was specified")
    
## Drop rows that have NA values.
# these corresponds to DCE experiments that 
# haven't been answered
df_scenario = df_scenario.dropna()
df_scenario.shape


(192, 16)

# Reformat columns and values

In [8]:
df_scenario['choice'] = (df_scenario.Alt_chosen.str.split(expand=True)[1]).replace({'A':1, 'B':2})

In [9]:
if current_summer: ## this scenario only has 4 variables
    alt_varying_variables = {'att_1': dict([(1, 'alt_1_att_1'),
                                            (2, 'alt_2_att_1')]),
                             'att_2': dict([(1, 'alt_1_att_2'),
                                            (2, 'alt_2_att_2')]),
                             'att_3': dict([(1, 'alt_1_att_3'),
                                            (2, 'alt_2_att_3')]),
                            'att_4': dict([(1, 'alt_1_att_4'),
                                            (2, 'alt_2_att_4')]),
                            }
    
else: ## the future scenarios have 6 variables
    alt_varying_variables = {'att_1': dict([(1, 'alt_1_att_1'),
                                        (2, 'alt_2_att_1')]),
                          'att_2': dict([(1, 'alt_1_att_2'),
                                        (2, 'alt_2_att_2')]),
                          'att_3': dict([(1, 'alt_1_att_3'),
                                        (2, 'alt_2_att_3')]),
                          'att_4': dict([(1, 'alt_1_att_4'),
                                        (2, 'alt_2_att_4')]),
                          'att_5': dict([(1, 'alt_1_att_5'),
                                        (2, 'alt_2_att_5')]),
                          'att_6': dict([(1, 'alt_1_att_6'),
                                        (2, 'alt_2_att_6')]),
                            }

ind_variables = []

availability_variables = {1: 'AV_0',
                          2: 'AV_1'}

In [10]:
## Transform wide to long format
# Note that long format is simply splitting
# each row into two, where each available alternative's attribute
# is on its own row, even if the alternative was not chosen
custom_alt_id = "alternative_id"

obs_id_column = "custom_id"
df_scenario[obs_id_column] = np.arange(df_scenario.shape[0],
                                            dtype=int) + 1

# specify the variable recording the choice column
choice_column = "choice"

df_long = pl.convert_wide_to_long(df_scenario, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
df_long.head()



/Users/hassanobeid/anaconda3/lib/python3.7/site-packages/pylogit/choice_tools.py:453: UserWarning: Note, there are 18 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 16 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,custom_id,alternative_id,choice,att_1,att_2,att_3,att_4,att_5,att_6
0,1,1,0,$200,3 degrees colder,4 degrees hotter,10% reduction,50% reduction,50% reduction
1,1,2,1,"$10,000",2 degrees colder,7 degrees hotter,25% reduction,70% reduction,No change
2,2,1,1,"$10,000",1 degree colder,3 degrees hotter,No change,No change,No change
3,2,2,0,"$7,000",2 degrees colder,7 degrees hotter,10% reduction,No change,15% reduction
4,3,1,1,$100,2 degrees colder,3 degrees hotter,50% reduction,50% reduction,15% reduction


In [11]:
## Change the data type of each column to numeric

df_long.att_1 = df_long.att_1.str.replace('$','').str.replace(',','').astype(int)
df_long.att_2 = df_long.att_2.str.replace(',','').str.split(expand=True)[0].replace({'No':0}).astype(int)
df_long.att_3 = df_long.att_3.str.replace(',','').str.split(expand=True)[0].replace({'No':0}).astype(int)

df_long.att_4 = df_long.att_4.str.replace(',','').str.split(expand=True)[0].replace(
    {'No':0}).str.strip('%').fillna(0).astype(int)/100

if (future_summer or future_winter):
    df_long.att_5 = df_long.att_5.str.replace(',','').str.split(expand=True)[0].replace(
        {'No':0}).str.strip('%').fillna(0).astype(int)/100
    df_long.att_6 = df_long.att_6.str.replace(',','').str.split(expand=True)[0].replace(
        {'No':0}).str.strip('%').fillna(0).astype(int)/100

df_long.head()

,custom_id,alternative_id,choice,att_1,att_2,att_3,att_4,att_5,att_6
0,1,1,0,200,3,4,0.10,0.5,0.50
1,1,2,1,10000,2,7,0.25,0.7,0.00
2,2,1,1,10000,1,3,0.00,0.0,0.00
3,2,2,0,7000,2,7,0.10,0.0,0.15
4,3,1,1,100,2,3,0.50,0.5,0.15


In [12]:
## Give meaningful variable names to the columns
if current_summer:
    variables = df_long.columns[-4:].to_list()  
    variables_names = ['Economic benefit', 'Before event decrease in temperature',
                  'During event increase in temperature', 
                  'During event reduction in artificial lighting']
elif future_summer:
    variables = df_long.columns[-6:].to_list()  
    variables_names = ['Economic benefit', 'Before event decrease in temperature',
                  'During event increase in temperature','During event reduction in air ventilation outdoor',
                  'During event reduction in artificial lighting', 'During event reduction in electricity usage']
elif future_winter:
    variables = df_long.columns[-6:].to_list()  
    variables_names = ['Economic benefit', 'Before event increase in temperature',
                  'During event decrease in temperature', 'During event reduction in air ventilation outdoor',
                  'During event reduction in artificial lighting', 'During event reduction in electricity usage']
else:
    print("Error: No analysis scenario was specified")

# Specify and fit the model

In [13]:
## Specify the DCM
basic_specification = OrderedDict()
basic_names = OrderedDict()

for i, var in enumerate(variables):
    basic_specification[var] = [[1,2]] ## Which utility equation does the variable enter
    basic_names[var] = [variables_names[i]]
    
basic_names

OrderedDict([('att_1', ['Economic benefit']),
             ('att_2', ['Before event decrease in temperature']),
             ('att_3', ['During event increase in temperature']),
             ('att_4', ['During event reduction in air ventilation outdoor']),
             ('att_5', ['During event reduction in artificial lighting']),
             ('att_6', ['During event reduction in electricity usage'])])

In [14]:
## Fit the DCM
estimates = MNL.fit_MNL(df_long,basic_specification,basic_names,'custom_id',
            'choice','alternative_id',np.zeros(len(variables)),const_pos = None)
# estimates.insert(0,'initial_betas',value=initial_betas)
estimates['confidence_lower_bound'] = estimates['Estimates']-1.96*estimates['std errors']
estimates['confidence_upper_bound'] = estimates['Estimates']+1.96*estimates['std errors']

estimates

Initial Log-likelihood:  -133.0843
Null Log-likelihood(LL0):-133.0843
Estimation time:  0.01
Final Log-likelihood:    -91.1167
Rho_bar sqrd =            0.3153


,Estimates,std errors,t_stat,p_values,confidence_lower_bound,confidence_upper_bound
Economic benefit,0.000264,0.000045,5.91,0.0000,0.000176,0.000351
Before event decrease in temperature,0.169620,0.125990,1.35,0.1770,-0.077321,0.416560
During event increase in temperature,-0.330120,0.072565,-4.55,0.0000,-0.472347,-0.187894
During event reduction in air ventilation outdoor,0.687740,0.694742,0.99,0.3222,-0.673955,2.049434
During event reduction in artificial lighting,-2.536045,0.587593,-4.32,0.0000,-3.687728,-1.384363
During event reduction in electricity usage,0.354802,0.671008,0.53,0.5961,-0.960374,1.669979


# Calculate and add elasticities to the results 

In [15]:
def calc_elasticity(util, b1, att1): 
    return b1*att1*(1/(1+np.exp(util)))

In [16]:
if current_summer: 
    b1 = estimates['Estimates'].loc['Economic benefit']
    b2 = estimates['Estimates'].loc['Before event decrease in temperature']
    b3 = estimates['Estimates'].loc['During event increase in temperature']
    b4 = estimates['Estimates'].loc['During event reduction in artificial lighting']

    betas=np.array([b1,b2,b3,b4]).reshape((4,1))

    marginal_df = df_long.loc[:,['att_1', 'att_2', 'att_3', 'att_4']]

    util = np.dot(marginal_df,betas)


    elast_b1 = calc_elasticity(util, b1, df_long['att_1'].values).mean()
    elast_b2 = calc_elasticity(util, b2, df_long['att_2'].values).mean()
    elast_b3 = calc_elasticity(util, b3, df_long['att_3'].values).mean()
    elast_b4 = calc_elasticity(util, b4, df_long['att_4'].values).mean()

    estimates['Average Elasticities'] = [elast_b1, elast_b2,
                                             elast_b3,elast_b4,
                                            ]
elif future_summer: 
    b1 = estimates['Estimates'].loc['Economic benefit']
    b2 = estimates['Estimates'].loc['Before event decrease in temperature']
    b3 = estimates['Estimates'].loc['During event increase in temperature']
    b4 = estimates['Estimates'].loc['During event reduction in air ventilation outdoor']
    b5 = estimates['Estimates'].loc['During event reduction in artificial lighting']
    b6 = estimates['Estimates'].loc['During event reduction in electricity usage']


    betas=np.array([b1,b2,b3,b4,b5,b6]).reshape((6,1))

    marginal_df = df_long.loc[:,['att_1', 'att_2', 'att_3', 'att_4', 'att_5', 'att_6']]

    util = np.dot(marginal_df,betas)

    elast_b1 = calc_elasticity(util, b1, df_long['att_1'].values).mean()
    elast_b2 = calc_elasticity(util, b2, df_long['att_2'].values).mean()
    elast_b3 = calc_elasticity(util, b3, df_long['att_3'].values).mean()
    elast_b4 = calc_elasticity(util, b4, df_long['att_4'].values).mean()
    elast_b5 = calc_elasticity(util, b5, df_long['att_5'].values).mean()
    elast_b6 = calc_elasticity(util, b6, df_long['att_6'].values).mean()



    estimates['Average Elasticities'] = [elast_b1, elast_b2,
                                             elast_b3,elast_b4,
                                            elast_b5, elast_b6]

    
elif future_winter: 
    b1 = estimates['Estimates'].loc['Economic benefit']
    b2 = estimates['Estimates'].loc['Before event increase in temperature']
    b3 = estimates['Estimates'].loc['During event decrease in temperature']
    b4 = estimates['Estimates'].loc['During event reduction in air ventilation outdoor']
    b5 = estimates['Estimates'].loc['During event reduction in artificial lighting']
    b6 = estimates['Estimates'].loc['During event reduction in electricity usage']


    betas=np.array([b1,b2,b3,b4,b5,b6]).reshape((6,1))

    marginal_df = df_long.loc[:,['att_1', 'att_2', 'att_3', 'att_4', 'att_5', 'att_6']]

    util = np.dot(marginal_df,betas)

    elast_b1 = calc_elasticity(util, b1, df_long['att_1'].values).mean()
    elast_b2 = calc_elasticity(util, b2, df_long['att_2'].values).mean()
    elast_b3 = calc_elasticity(util, b3, df_long['att_3'].values).mean()
    elast_b4 = calc_elasticity(util, b4, df_long['att_4'].values).mean()
    elast_b5 = calc_elasticity(util, b5, df_long['att_5'].values).mean()
    elast_b6 = calc_elasticity(util, b6, df_long['att_6'].values).mean()



    estimates['Average Elasticities'] = [elast_b1, elast_b2,
                                             elast_b3,elast_b4,
                                            elast_b5, elast_b6]

estimates

,Estimates,std errors,t_stat,p_values,confidence_lower_bound,confidence_upper_bound,Average Elasticities
Economic benefit,0.000264,0.000045,5.91,0.0000,0.000176,0.000351,0.549826
Before event decrease in temperature,0.169620,0.125990,1.35,0.1770,-0.077321,0.416560,0.155251
During event increase in temperature,-0.330120,0.072565,-4.55,0.0000,-0.472347,-0.187894,-0.824318
During event reduction in air ventilation outdoor,0.687740,0.694742,0.99,0.3222,-0.673955,2.049434,0.083872
During event reduction in artificial lighting,-2.536045,0.587593,-4.32,0.0000,-3.687728,-1.384363,-0.556963
During event reduction in electricity usage,0.354802,0.671008,0.53,0.5961,-0.960374,1.669979,0.043542
